# Simple Lineare Regression mit `statsmodels` berechnen

In [6]:
# Import dstools (absolute path required, please change to your systems settings)
import importlib
import sys

path = '/dstools-master/dstools/__init__.py'
name = 'dstools'

spec = importlib.util.spec_from_file_location(name, path)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module
spec.loader.exec_module(module)

In [7]:
import pandas as pd
import numpy as np
import statsmodels as sm
import statsmodels.formula.api as smf

from dstools.datasets import sunshine

In [9]:
# Datensatz laden
import csv
df = pd.read_csv('advertising.csv')
print(df)

      ID     TV  Radio  Newspaper  Sales
0      1  230.1   37.8       69.2   22.1
1      2   44.5   39.3       45.1   10.4
2      3   17.2   45.9       69.3    9.3
3      4  151.5   41.3       58.5   18.5
4      5  180.8   10.8       58.4   12.9
..   ...    ...    ...        ...    ...
195  196   38.2    3.7       13.8    7.6
196  197   94.2    4.9        8.1    9.7
197  198  177.0    9.3        6.4   12.8
198  199  283.6   42.0       66.2   25.5
199  200  232.1    8.6        8.7   13.4

[200 rows x 5 columns]


## Response- und Predictorvariable festlegen

Eine bequemere Art die lineare Regression durchzuführen ist das Paket `statsmodels` zu verwenden, wie schon die wesentlich kompaktere und übersichtlichere Syntax zeigt. Mit `smf` können wir eine direkte Formelschreibweise (wie es auch in R möglich ist) verwenden.

In [10]:
results = smf.ols('TV ~ Sales', data=df).fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                     TV   R-squared:                       0.612
Model:                            OLS   Adj. R-squared:                  0.610
Method:                 Least Squares   F-statistic:                     312.1
Date:                Thu, 31 Oct 2024   Prob (F-statistic):           1.47e-42
Time:                        14:41:27   Log-Likelihood:                -1079.2
No. Observations:                 200   AIC:                             2162.
Df Residuals:                     198   BIC:                             2169.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -33.4502     10.897     -3.070      0.0

## Zusammenfassung der Werte

Im Vergleich mit der Zusammenfassung der manuellen Berechnung sehen wir großteils Übereinstimmung. Etwas später werden wir die Differenz bei AIC und BIC erklären.

In [15]:
print("beta_0\t\t", results.params[0])
print("beta_1\t\t", results.params[1])
print("t beta_0\t", results.tvalues[0])
print("t beta_1\t", results.tvalues[1])
print("p beta_0\t", results.pvalues[0])
print("p beta_1\t", results.pvalues[1])
print("CI beta0 lo\t", results.conf_int().values[0][0])
print("CI beta0 hi\t", results.conf_int().values[0][1])
print("CI beta1 lo\t", results.conf_int().values[1][0])
print("CI beta1 hi\t", results.conf_int().values[1][1])
print("R^2\t\t", results.rsquared)
print("adj. R^2\t", results.rsquared_adj)
print("F-Stat\t\t", results.fvalue)
print("LogLik\t\t", results.llf)
print("AIC\t\t", results.aic)
print("BIC\t\t", results.bic)

beta_0		 15.728319304914468
beta_1		 5.3364105348900335
t beta_0	 3.5446280536005523
t beta_1	 5.601184467186704
p beta_0	 0.012150695835125902
p beta_1	 0.001379362627713965
CI beta0 lo	 4.870815972034617
CI beta0 hi	 26.58582263779432
CI beta1 lo	 3.0051668029806353
CI beta1 hi	 7.667654266799431
R^2		 0.8394574407934108
adj. R^2	 0.8127003475923126
F-Stat		 31.373267435453595
LogLik		 -22.358617621507904
AIC		 48.71723524301581
BIC		 48.876118326375476


C:\Users\a3445\AppData\Local\Temp\ipykernel_768\1259289454.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("beta_0\t\t", results.params[0])
C:\Users\a3445\AppData\Local\Temp\ipykernel_768\1259289454.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("beta_1\t\t", results.params[1])
C:\Users\a3445\AppData\Local\Temp\ipykernel_768\1259289454.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("t beta_0\t", resul

# Gemeinsamkeiten und Unterschiede zu R

Wir sehen uns nun die lineare Regression in R an:

<a href="http://rstudio.bds.fhstp.ac.at" target="new">R Studio Server Pro</a>

## Residuen

Sehen wir uns an, wie Python standardmässig die Quartile der Residuen ausgibt. Dazu sortieren wir sie zunächst, um ein besseres Verständnis zu haben:

In [7]:
residuals = results.resid

In [8]:
residuals.values.sort(axis=0)
residuals

0   -5.213141
1   -3.867499
2   -2.809449
3   -2.804833
4    0.849525
5    3.921423
6    3.930654
7    5.993321
dtype: float64

In [9]:
np.quantile(residuals,[.0,.25,.5,.75,1])

array([-5.21314146, -3.07396144, -0.97765409,  3.92373065,  5.99332066])

Die standardmässige Ausgabe liefert das gleiche Ergebnis wie der Standard bei R. Aber auch in Python gibt es mehrere Optionen. Die Option `type=2` aus R ist in Python `interpolation='midpoint'`:

In [10]:
np.quantile(residuals,[.25,.5,.75], method='midpoint') # R type=2,5

array([-3.33847407, -0.97765409,  3.92603856])

## AIC, BIC in Python Statsmodel

Statsmodel hat eine andere Art das AIC zu berechnen. In `linear_model.py` ist die Funktion `aic()` definiert als

```python
    def aic(self):
        return -2 * self.llf + 2 * (self.df_model + self.k_constant)
```

wobei `df.model` die Freiheitsgrade des Modells sind (also in dem Fall 1, da wir nur eine unabhängige Variable haben) und `k.constant` ist für die lineare Regression = 1. `llf` ist der LogLikelihood Wert und stimmt mit dem von R überein. Wir haben daher als Unterschied: 

$$AIC_R = -2\cdot LL + 2 \cdot 3$$
$$AIC_{Py} = -2 \cdot LL + 2 \cdot 2$$

Das `self.df_model` wird berechnet als

```python
self._df_model = float(self.rank - self.k_constant)
```

wobei hier `rank` der Rang der Matrix der Singulärwerte von $X$ ist, in unserem Fall = 2. Würde Python für das AIC statt `self.df_model` den Wert `df.rank` verwenden, hätte man das gleiche Ergebnis wie in R.